In [1]:
import os
import numpy as np
import pandas as pd
from numba import jit
from tqdm import tqdm
import statsmodels.api as sm
import sys
sys.path.append('D:\实习\研报复现\QuantDev\ymtool')
import Alpha as alpha

import warnings
warnings.filterwarnings('ignore')

# 价量相关性因子的逻辑

In [4]:
def pv_corr_func(date):
    close_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\close",date)
    volume_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\volume",date)
    date = pd.to_datetime(date.split('.')[0],format='%Y%m%d')
    close = pd.read_parquet(close_path)
    volume = pd.read_parquet(volume_path)
    code = close.columns
    close = close.values
    volume = volume.values

    cov = np.mean((close-np.mean(close,axis=0))*(volume-np.mean(volume,axis=0)),axis=0)
    pv_corr = cov/np.std(close,axis=0)/np.std(volume,axis=0)
    pv_corr = pd.Series(pv_corr,index=code, name=date)
    
    return pv_corr
    
data = os.listdir(r"D:\实习\研报复现\dataAll\stockMinData\close")



100%|██████████████████████████████████████████████████████████████████████████████| 2512/2512 [19:54<00:00,  2.10it/s]


In [ ]:
pv_corr = []

for i in tqdm(data):
    
    pv_corr_ = pv_corr_func(date=i)
    pv_corr.append(pv_corr_)
    
pv_corr = pd.concat(pv_corr, axis=1).T

In [5]:
ret20 = alpha.load_data('pv\\adj_close_return').rolling(20).sum().stack()
def deRet20(fct):
    res = []
    for idx in tqdm(fct.index):
        temp = pd.concat([fct.loc[idx],ret20.loc[idx]],axis=1).dropna()

        X = temp.iloc[:,1:]
        X = sm.add_constant(X)
        y = temp.iloc[:,0]

        # 创建模型
        model = sm.OLS(y, X)

        # 拟合模型
        results = model.fit()

        res.append(results.resid)

    fctNeu = pd.concat(res,axis=1).T
    fctNeu.index = fct.index
    return fctNeu

In [8]:
pv_corr_avg = deRet20(alpha.mvind_neutral(pv_corr.replace([np.inf,-np.inf],0).rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03']))
pv_corr_std = deRet20(alpha.mvind_neutral(pv_corr.replace([np.inf,-np.inf],0).rolling(20,min_periods=1).std().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03']))

100%|█████████████████████████████████████████████████████████████████████████████| 2472/2472 [00:13<00:00, 176.62it/s]


In [9]:
pv_corr_det20 = (alpha.zscore(pv_corr_avg)+alpha.zscore(pv_corr_std)).dropna(axis=0,how='all')

# 价量相关性的趋势因子

In [11]:
Beta = []
for i in tqdm(range(20,len(pv_corr)+1)):
    x = np.array(np.arange(1,21)).reshape(20,1)

    beta = np.linalg.pinv(x.T @ x) @ x.T @ (pv_corr.iloc[i-20:i].dropna(axis=1))
    Beta.append(beta.T)

Beta = pd.concat(Beta,axis=1).T

Beta.index = pv_corr.index[19:]

100%|█████████████████████████████████████████████████████████████████████████████| 2493/2493 [00:04<00:00, 558.78it/s]


In [13]:
ret20 = alpha.load_data('pv\\adj_close_return').rolling(20,min_periods=1).sum().stack()
std20 = alpha.load_data('pv\\adj_close_return').rolling(20,min_periods=1).std().stack()
tnv20 = alpha.load_data('pv\\stock_turnover').rolling(20,min_periods=1).mean().stack()

def depv(fct):
    res = []
    for idx in tqdm(fct.index):
        temp = pd.concat([fct.loc[idx],ret20.loc[idx],\
                          std20.loc[idx],tnv20.loc[idx]],axis=1).dropna()
        
        X = temp.iloc[:,1:]
        X = sm.add_constant(X)
        y = temp.iloc[:,0]

        # 创建模型
        model = sm.OLS(y, X)

        # 拟合模型
        results = model.fit()

        res.append(results.resid)

    fctNeu = pd.concat(res,axis=1).T
    fctNeu.index = fct.index
    return fctNeu

In [14]:
pv_corr_trend = alpha.mvind_neutral(depv(Beta.loc[:'2024-03']).dropna(axis=0,how='all'))

100%|██████████████████████████████████████████████████████████████████████████████| 2473/2473 [00:39<00:00, 63.26it/s]


# CPV

In [15]:
CPV = alpha.zscore(pv_corr_trend)+alpha.zscore(pv_corr_det20)

In [20]:
alpha.to_fctDataBase(CPV.dropna(axis=0,how='all'),'CPV')